In [140]:
import requests
import io
from PIL import Image
import base64
from dash import Dash, html, dcc, Output, Input

# Initialize app
app = Dash(__name__)
server = app.server  # for deployment later if you want

# Constants
URL = "https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi"
# BBOX = "-10.8673,-64.4373,-8.6430,-61.3161"
BBOX = "-10.8673,-65.9979,-8.6430,-59.7555"
def get_image_base64(layer, date_str):
        params = {
            "service": "WMS",
            "version": "1.3.0",
            "request": "GetMap",
            "layers": layer,
            "style": "default",
            "format": "image/png",
            "transparent": "false",
            "crs": "EPSG:4326",
            "time": date_str,
            "bbox": BBOX,
            "width": "694",
            "height": "495",
        }
        r = requests.get(URL, params=params)
        r.raise_for_status()
        img = Image.open(io.BytesIO(r.content))
        buf = io.BytesIO()
        img.save(buf, format="PNG")
        encoded = base64.b64encode(buf.getvalue()).decode("ascii")

        return f"data:image/png;base64,{encoded}"

def year_slider(layer, start_year, end_year, title):

    YEARS = list(range(start_year, end_year + 1))

    app.layout = html.Div([
        html.H3(title),
        dcc.Slider(
            min=min(YEARS),
            max=max(YEARS),
            step=1,
            value=min(YEARS),
            marks={y: str(y) for y in YEARS},
            id="year-slider"
        ),
        html.Div(id="date-label", style={"margin": "10px", "font-weight": "bold"}),
        html.Img(id="landcover-image", style={"width": "700px", "border": "1px solid #ccc"})
    ])

    @app.callback(
        [Output("landcover-image", "src"), Output("date-label", "children")],
        [Input("year-slider", "value")]
    )
    def update_image(year):
        date_str = f"{year}-01-01"
        print(f"Fetching {date_str}")
        return get_image_base64(layer, date_str), f"Year: {year}"

    if __name__ == "__main__":
        app.run(debug=True)



In [141]:
# https://worldview.earthdata.nasa.gov/?l=Reference_Labels_15m(hidden),Reference_Features_15m(hidden),Coastlines_15m,MODIS_Combined_L3_IGBP_Land_Cover_Type_Annual,MODIS_Terra_CorrectedReflectance_TrueColor&lg=false&t=2001-01-02-T00%3A00%3A00Z
# year_slider('MODIS_Combined_L3_IGBP_Land_Cover_Type_Annual', 2005, 2015, title="MODIS Land Cover Type (L3, IGBP, Yearly)")

In [142]:
# https://worldview.earthdata.nasa.gov/?l=Reference_Labels_15m(hidden),Reference_Features_15m(hidden),Coastlines_15m,MODIS_Terra_L4_LAI_8Day,MODIS_Terra_CorrectedReflectance_TrueColor&lg=false&t=2000-02-27-T00%3A00%3A00Z
# year_slider('MODIS_Aqua_L4_LAI_8Day', 2005, 2015, title="MODIS Leaf Area Index (L4, 8-Day)")

In [143]:
# https://worldview.earthdata.nasa.gov/?l=Reference_Labels_15m(hidden),Reference_Features_15m(hidden),Coastlines_15m,MODIS_Aqua_L3_NDVI_16Day,MODIS_Terra_CorrectedReflectance_TrueColor&lg=false&t=2002-07-05-T00%3A00%3A00Z
# year_slider(layer = 'MODIS_Aqua_L3_NDVI_16Day', start_year = 2015, end_year = 2025, title = "MODIS NDVI (L3, 16-Day)")